# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [ ]:
!apt-get update && apt-get install -y python3-opencv

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
!pip install sklearn

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Feb 13 14:19:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [110]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from pylab import rcParams
from google.colab import drive
drive.mount('/content/drive/')
from google.colab.patches import cv2_imshow 
import torchvision.transforms.functional as TF
from imgaug import augmentables
import imgaug as ia
import imgaug.augmenters as iaa
import torchvision


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Set Arguments & hyperparameters

In [111]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [143]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= '/content/drive/MyDrive/이어드림/project2/data'
NUM_CLS = 2
EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.0005
EARLY_STOPPING_PATIENCE = 20
INPUT_SHAPE = 128
os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda')
DATA_SIZE = 1000
TRAIN_PER = 0.85

## Dataloader

#### Train & Validation Set loader

In [ ]:
# !unzip -uq '/content/drive/MyDrive/이어드림/project2/test.zip' -d '/content/drive/MyDrive/이어드림/project2'
# !unzip -uq '/content/drive/MyDrive/이어드림/project2/train.zip' -d '/content/drive/MyDrive/이어드림/project2'


In [144]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape, data_size, train_per):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        self.data_size = data_size
        self.train_per = train_per
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int((self.data_size) * (self.train_per))]
        elif self.mode == 'val':
            self.db = self.db[int((self.data_size) * (self.train_per)):int(self.data_size)]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        def dbplus(db):
            augdb = db.copy()
            for i in range(51):
                if i == 0:
                    augdb['file_name'] = db['file_name'].apply(lambda x : int(x[:-4]) + int('646'))
                    augdb['file_name'] = augdb['file_name'].apply(lambda x : str(x) +'.png')
                    db = pd.concat([db, augdb], ignore_index = True,axis=0)
                else :
                    augdb['file_name'] = augdb['file_name'].apply(lambda x : int(x[:-4]) + int('646'))
                    augdb['file_name'] = augdb['file_name'].apply(lambda x : str(x) +'.png')
                    db = pd.concat([db, augdb], ignore_index = True,axis=0)
            return db
        return dbplus(db)
        
    def __len__(self):
        if self.mode == 'train':
            return int((int(self.data_size))*(self.train_per))
        elif self.mode == 'val':
            return int((int(self.data_size)) - int((int(self.data_size))*(self.train_per)))
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))
         
        return trans_image, data['COVID']

    

In [107]:
#augment시 사용
eq_img_list = []
class Augmentation:
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        self.img = None
    
        #이미지크기 저장공간

        self.db = self.data_loader()
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
       
        return db
    def Equalization(self):
    #    // for cnt in tqdm(range(646)):
        for i in tqdm(range(646)):
            # img = eq_img_list
            db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
            data = copy.deepcopy(db.loc[i])
            cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']))
            cvimg_yuv = cv2.cvtColor(cvimg, cv2.COLOR_BGR2YUV)

            img_clahe = cvimg_yuv.copy()
            clahe = cv2.createCLAHE(clipLimit = 3.0 , tileGridSize=(8,8))
            img_clahe[:,:,0] = clahe.apply(img_clahe[:,:,0])
            img_clahe = cv2.cvtColor(img_clahe, cv2.COLOR_YUV2BGR)

                # img_eq = cv2_imshow(img_clahe)
            # cv2.waitKey()
            cv2.destroyAllWindows()
            eq_img_list.append(img_clahe)
            # os.makedirs('/content/drive/MyDrive/이어드림/project2/data/aug_img', exist_ok=True)
            cv2.imwrite(f'/content/drive/MyDrive/이어드림/project2/data/train/{i}.png', img_clahe)
            self.img = eq_img_list
        return eq_img_list

    def augmentation(self):
        
        for i in tqdm(range(646)):
            
            db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
            data = copy.deepcopy(db.loc[i])
            sss =cv2.imread(os.path.join(self.data_dir,'train',data['file_name']))
            
            # cv2_imshow(sss)
            augmentation_resize = iaa.Sequential([
                            iaa.Resize({"height":384,"width":384},interpolation="cubic")
                        ])


            image_aug = augmentation_resize(image=sss)
            # cv2_imshow(image_aug)
            # sometimes = lambda aug: iaa.Sometimes(0.5, aug)
            for j in (range(10)):
                augmentation_resize = iaa.Sequential([
                                    # iaa.Affine(translate_percent={"x":(-0.5,0.5),"y":(-0.5,0.5)},rotate=(-2,2),scale=(0.5,2)),
                                    # iaa.Crop(px=(0, 16)),
                                    # iaa.Multiply((0.8, 1.3)),
                                    iaa.LinearContrast((0.75,1.45)),
                                    iaa.GaussianBlur((0.0,1.5)),
                                    iaa.Fliplr(0.5),
                                    iaa.Flipud(0.3),
                                    # iaa.Affine(translate_px={"x": (-20, 20), "y": (-20, 20)}),
                                    # iaa.PerspectiveTransform(scale=(0.01, 0.15)),
                                    # iaa.PiecewiseAffine(scale=(0.01, 0.05)),
                                    iaa.Sharpen(alpha=(0, 0.3), lightness=(0.75, 1.00)),
                                    # iaa.Emboss(alpha=(0, 1.0), strength=(0, 1.5)),
                                     
                                    ])
 
                image_augs = augmentation_resize(image=image_aug)
            # cv2_imshow(image_augs)
                # os.makedirs('/content/drive/MyDrive/이어드림/project2/data/augment{}'.format(j), exist_ok=True)
                #cv2.imwrite(f'/content/drive/MyDrive/이어드림/project2/data/train/{(i)+(1292)+(646*j)}.png', image_augs)
                cv2.imwrite(f'/content/drive/MyDrive/이어드림/project2/data/train/{(i)+(646)+(646*j)}.png', image_augs)


In [108]:
#data augm
if __name__ == '__main__': 
    temp = Augmentation(DATA_DIR,'train', INPUT_SHAPE)
    print('이미w지를 불러오는 중입니다.')
    temp.data_loader()
    print('*' * 30)
    print('Equalization중 입니다')
    # temp.Equalization()
    print('*' * 30)
    print('Equalization 성공')
    print('Augmentation중 입니다')
    temp.augmentation()
    print('*' * 30)
    del temp
    

Loading train dataset..
이미w지를 불러오는 중입니다.
Loading train dataset..
******************************
Equalization중 입니다
******************************
Equalization 성공
Augmentation중 입니다


100%|██████████| 646/646 [51:27<00:00,  4.78s/it]

******************************


## Model

In [13]:
# import torch.nn.functional as F

# class custom_CNN(nn.Module):
#     def __init__(self, num_classes):
#         super(custom_CNN, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
#         self.bn1 = nn.BatchNorm2d(8)
#         self.conv2 = nn.Conv2d(in_channels=8, out_channels=25, kernel_size=5)
#         self.bn2 = nn.BatchNorm2d(25)
#         self.pool = nn.MaxPool2d(kernel_size=2)

#         self.fc1 = nn.Linear(in_features=25*29*29, out_features=128)
#         self.fc2 = nn.Linear(in_features=128, out_features=num_classes)

#         self.softmax = nn.Softmax(dim=1)
        
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x))) # (32, 3, 128, 128) -> (32, 8, 62, 62)
#         x = self.pool(F.relu(self.conv2(x))) # (32, 8, 62, 62) -> (32, 25, 29, 29)
        
#         x = torch.flatten(x,1)
#         x = self.conv
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))

#         output = self.softmax(x)
        
#         return output
import torch.nn.functional as F

class custom_CNN(nn.Module):
    def __init__(self, num_classes):
        super(custom_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(24)
        self.fc1 = nn.Linear(864, 128)
        self.fc2 = nn.Linear(in_features=128, out_features=num_classes)
        self.softmax = nn.Softmax(dim=1)
        self.dropout = F.dropout(p=0.3)
    def forward(self, input):
        output = F.relu(self.bn1(self.conv1(input)))  
        output = self.pool(output)    
        output = F.relu(self.bn2(self.conv2(output)))     
        output = self.pool(output)                        
        output = F.relu(self.bn4(self.conv4(output)))     
        output = self.pool(output)
        output = F.relu(self.bn5(self.conv5(output)))    
        output = self.pool(output) 
        # output = output.view(-1, 24*10*10)/
        output = torch.flatten(output,1)
        output = self.fc1(output)
        
        output = self.fc2(output)
        output = self.softmax(output)

        return output


In [145]:
import torch.nn.functional as F

class custom_CNN(nn.Module):
    def __init__(self, num_classes):
        super(custom_CNN, self).__init__()
        self.layer1=nn.Sequential(
                        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2,stride=2),
                        nn.BatchNorm2d(32)
                    )
        
        self.layer2=nn.Sequential(
                        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2,stride=2),
                        nn.BatchNorm2d(32)
                    )
        
        self.layer3=nn.Sequential(
                        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2,stride=2),
                        nn.BatchNorm2d(64)
                    )
        
        self.layer4=nn.Sequential(
                        nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2,stride=2),
                        nn.BatchNorm2d(64)
                    )
        
        self.fc1 = nn.Sequential(
                        nn.Linear(in_features=64*14*14, out_features=64*14),
                        nn.Dropout(p=0.3)
                    )
        self.fc2 = nn.Sequential(
                        nn.Linear(in_features=64*14, out_features=128),
                        nn.Dropout(p=0.3)
                    )
        self.fc3 = nn.Sequential(
                        nn.Linear(in_features=128, out_features=num_classes),
                        nn.Dropout(p=0.3)
                    )
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        # print(x.shape)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        
        output = self.softmax(x)
        # print(output)
        return output

if __name__ == '__main__':
    x=torch.randn(5,3,128,128)
    model=custom_CNN(2)

    y=model(x)
    print(y)
    print(y.shape)

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<SoftmaxBackward0>)
torch.Size([5, 2])


## Utils
### EarlyStopper

In [146]:
class LossEarlyStopper():


    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [147]:
class Trainer():

    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
       
    
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [148]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [149]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE, data_size= DATA_SIZE, train_per = TRAIN_PER)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE, data_size = DATA_SIZE, train_per = TRAIN_PER)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 850 Val set samples: 150


TypeError: ignored

#### Load model and other utils

In [150]:
# Load Model
model = custom_CNN(NUM_CLS).to(DEVICE)
model.train()

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

In [123]:
model

custom_CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

In [139]:
model

custom_CNN(
  (conv1): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 25, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=21025, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

### epoch 단위 학습 진행

In [151]:
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.6833958786267501, Acc: 0.5952941176470589, F1-Macro: 0.590283382656264


  2%|▏         | 1/50 [00:08<06:41,  8.19s/it]

Epoch 0, Val loss: 0.751901850104332, Acc: 0.7466666666666667, F1-Macro: 0.7311827956989247
Epoch 1, Train loss: 0.5601236327336385, Acc: 0.7105882352941176, F1-Macro: 0.7027532638857299
Epoch 1, Val loss: 0.6215328797698021, Acc: 0.78, F1-Macro: 0.776412665432043
Validation loss decreased 0.751901850104332 -> 0.6215328797698021


  4%|▍         | 2/50 [00:16<06:38,  8.30s/it]

...

Epoch 26, Train loss: 0.2154231733427598, Acc: 0.8717647058823529, F1-Macro: 0.867585919088065
Epoch 26, Val loss: 0.8512559086084366, Acc: 0.8533333333333334, F1-Macro: 0.8516453875202301
Early stopping counter 19/20


 54%|█████▍    | 27/50 [03:50<03:15,  8.50s/it]

Epoch 27, Train loss: 0.20798907400323793, Acc: 0.8623529411764705, F1-Macro: 0.8573618905967852


 54%|█████▍    | 27/50 [03:58<03:23,  8.84s/it]

Epoch 27, Val loss: 0.9050202406942844, Acc: 0.8533333333333334, F1-Macro: 0.8520444763271162
Early stopping counter 20/20
Early stopped


## Inference
### 모델 로드

In [152]:
TRAINED_MODEL_PATH = 'best.pt'


TypeError: ignored

### Load dataset

In [153]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [154]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행

In [155]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:00,  3.96it/s]

tensor([[9.2367e-01, 7.6334e-02],
        [1.6182e-01, 8.3818e-01],
        [8.2463e-01, 1.7537e-01],
        [1.3793e-03, 9.9862e-01],
        [9.8489e-01, 1.5108e-02],
        [1.8971e-06, 1.0000e+00],
        [1.2099e-02, 9.8790e-01],
        [9.4798e-01, 5.2017e-02],
        [9.3026e-01, 6.9737e-02],
        [6.1402e-10, 1.0000e+00],
        [9.0248e-01, 9.7517e-02],
        [9.9131e-01, 8.6939e-03],
        [9.9576e-01, 4.2380e-03],
        [1.0000e+00, 2.6981e-06],
        [6.8009e-04, 9.9932e-01],
        [9.7764e-01, 2.2357e-02],
        [1.8303e-02, 9.8170e-01],
        [9.6611e-01, 3.3886e-02],
        [9.3632e-01, 6.3676e-02],
        [6.2616e-01, 3.7384e-01],
        [9.9922e-01, 7.8250e-04],
        [9.9640e-01, 3.5985e-03],
        [3.6933e-03, 9.9631e-01],
        [9.9344e-01, 6.5628e-03],
        [3.5843e-01, 6.4157e-01],
        [9.9998e-01, 1.9002e-05],
        [9.3410e-01, 6.5904e-02],
        [3.5843e-03, 9.9642e-01],
        [5.3519e-05, 9.9995e-01],
        [1.211

2it [00:00,  3.90it/s]

tensor([[8.1598e-01, 1.8402e-01],
        [1.4985e-02, 9.8502e-01],
        [9.8757e-01, 1.2429e-02],
        [9.4490e-01, 5.5103e-02],
        [9.8330e-01, 1.6697e-02],
        [1.7062e-02, 9.8294e-01],
        [3.8219e-03, 9.9618e-01],
        [3.1828e-03, 9.9682e-01],
        [1.3423e-08, 1.0000e+00],
        [4.0939e-01, 5.9061e-01],
        [8.4563e-01, 1.5437e-01],
        [1.4123e-04, 9.9986e-01],
        [1.1714e-02, 9.8829e-01],
        [1.4079e-03, 9.9859e-01],
        [9.6711e-01, 3.2889e-02],
        [9.9990e-01, 1.0301e-04],
        [9.6801e-01, 3.1995e-02],
        [1.5209e-01, 8.4791e-01],
        [1.3517e-01, 8.6483e-01],
        [1.0785e-06, 1.0000e+00],
        [9.0879e-03, 9.9091e-01],
        [9.8209e-02, 9.0179e-01],
        [3.4521e-04, 9.9965e-01],
        [1.1498e-01, 8.8502e-01],
        [7.9316e-04, 9.9921e-01],
        [1.1274e-06, 1.0000e+00],
        [9.9999e-01, 1.0827e-05],
        [5.8278e-01, 4.1722e-01],
        [1.1065e-04, 9.9989e-01],
        [9.990

4it [00:00,  5.02it/s]

tensor([[4.1111e-02, 9.5889e-01],
        [1.0658e-08, 1.0000e+00],
        [4.1222e-05, 9.9996e-01],
        [9.9696e-01, 3.0388e-03],
        [2.2490e-01, 7.7510e-01],
        [9.3139e-01, 6.8612e-02],
        [9.1986e-01, 8.0135e-02],
        [9.5604e-01, 4.3964e-02],
        [9.9849e-01, 1.5128e-03],
        [9.9987e-01, 1.2750e-04],
        [9.0754e-04, 9.9909e-01],
        [9.9999e-01, 9.4578e-06],
        [9.9986e-01, 1.4352e-04],
        [9.9958e-01, 4.2194e-04],
        [4.2669e-03, 9.9573e-01],
        [9.6850e-05, 9.9990e-01],
        [1.7342e-04, 9.9983e-01],
        [9.8882e-01, 1.1184e-02],
        [9.9153e-01, 8.4732e-03],
        [1.4866e-06, 1.0000e+00],
        [6.7741e-05, 9.9993e-01],
        [6.7109e-01, 3.2891e-01],
        [9.0905e-03, 9.9091e-01],
        [9.9065e-01, 9.3492e-03],
        [9.9351e-01, 6.4891e-03],
        [6.8170e-04, 9.9932e-01],
        [4.7224e-01, 5.2776e-01],
        [9.9913e-01, 8.6691e-04],
        [2.3909e-02, 9.7609e-01],
        [6.378

### 결과 저장

In [156]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv(DATA_DIR+'/prediction(custom+1000,0.8).csv', index=False)
#과적합줄이기